In [ ]:
# visionneuse externe interactive
%matplotlib 

#imports
import numpy as np # tableaux
import scipy.ndimage as scp # convolution, transformée de distance
import matplotlib.pyplot as plt # visionneur et tracés

import skimage
from skimage import color # gestion de la couleur
from skimage import data # petite banque d'images
from skimage import io # lecture et sauvegarde d'images
from skimage import util # conversions entier <-> flottants etc.
from skimage import filters as flt # filtres, seuillage

# chemin d'accès à mes images 
my_path = "Images/Tmp/" 

In [ ]:
I = 1 - skimage.io.imread('Images/cheval.pbm')//255

# vérification qu'on a bien une image binaire
print(I.dtype, np.amax(I))

plt.imshow(I, cmap='gray')

## SKIZ

```
En_attente <- vide
Pour chaque marqueur Mi,
| initier la Zone d’influence IZi avec Mi
| mettre Mi dans la file FAi
Répéter jusqu’à stabilité :
| Pour chaque FAi
| | Pour chaque p dans FAi
| | | retirer p de FAi
| | | Pour chaque voisin q de p non encore étiqueté
| | | | Si q pas dans En_attente
| | | | | ajouter (q,i) à En_attente
| | | | Sinon si (q,j) dans En_attente avec j <> i,
| | | | | ajouter q au SKIZ
| | | | | retirer q de En_attente
| Pour chaque (p,i) dans En_attente
| | retirer p de En_attente et l’ajouter à FAi et à IZi
```

In [ ]:
def skiz(markers, topology, shape):
    '''
    Retourne la liste des coordonnées des pixels du SKIZ.
    
    :param markers: liste des pixels des marqueurs (liste de sets)
    :param topologie: entier donnant le type d'adjacence (4 ou 8)
    :param shape: couple d'entier (valeur de l'attribut shape de l'image)
    '''
      
    waiting = []
    n = len(markers)
    waiting_line = []
    influence_zone = []
    for i in range(n):
        waiting_line.append( markers[i].copy() )
        influence_zone.append( markers[i].copy() )
    skiz = []

    stability = False
    while not stability:
        for i in range(n):
            for p in waiting_line[i]:
                for q in neighbor(p, topology, shape):
                    environment = [p, i, influence_zone, skiz]
                    if isfree(q, environment):
                        if notfound(q, waiting):
                            waiting.append((q, i))
                        elif found(q, i, waiting):
                            skiz.append(q)
                            remove_all(q, waiting)
            waiting_line[i] = set()
        for (p, i) in waiting:
            waiting_line[i].add(p)
            influence_zone[i].add(p)
        waiting = []
        stability = are_empty(waiting_line)
    return skiz, influence_zone


def neighbor(p, topology, shape):
    '''
    Renvoie un set contenant les voisins de p pour la topologie considérée
    (4 ou 8 adjacence). Le paramètre shape permet de gérer les bords de l'image.
    '''
    i, j = p[0], p[1]
    height, width = shape
    _i, i_ = max( i-1, 0 ), min( i+1, height-1)
    _j, j_ = max( j-1, 0 ), min( j+1, width-1)
    neighborhood = {(i,_j), (_i,j), (i_,j), (i,j_)}
    if topology == 8:
        neigh8 = {(_i,_j), (i_,_j), (_i,j_), (i_,j_)}
        neighborhood |= neigh8
    if (i,j) in neighborhood:
        neighborhood.remove((i,j))
    return neighborhood
    
    
def isfree( q, environment):
    '''
    renvoie True si q n'est dans aucune zone d'influence ni dans le SKIZ,
    sinon renvoie False. De plus, lorsque q est déjà dans une zone d'influence
    dont l'indice n'est pas celui courant (cas du SKIZ épais), q est retiré de 
    cette zone et ajouté au SKIZ en même temps que le point p courant dont q est
    le voisin.
    
    '''
    [p, marker_index, zones, skeleton] = environment
    if q in skeleton:
        return False
    n = len(zones)
    for i in range(n):
        if q in zones[i]:
            if i != marker_index:
                zones[i].remove(q)
                skeleton.append(p)
                skeleton.append(q)
            return False      
    return True


def notfound(p, alist):
    '''
    Renvoie True ssi il n'existe pas de paire (p,i) dans alist.
    '''
    for (q,j) in alist:
        if q == p:
            return False
    return True

    
def found(p, i, alist):
    '''
    Renvoie True ssi il existe une paire (p,j) avec i<>j dans alist.
    '''
    for (q,j) in alist:
        if q == p and not j == i:
            return True
    return False


def remove_all(p, alist):
    '''
    Supprime toutes les paires de alist dont le premier élément est p.
    
    :param alist: liste de paires
    '''
    for (q,i) in alist:
        if q == p:
            alist.remove((q,i))

            
def are_empty(listoflist):
    '''
    Renvoie True ssi toutes les listes contenues dans listoflist sont vides.
    '''
    n = len(listoflist)
    for i in range(n):
        if len(listoflist[i]) > 0:
            return False
    return True

skel, iz = skiz( [ {(0,0)}, {(6,2)}, {(1,8), (0,8)}, {(8,14), (8,15)} ], 4, (9,16) )

img = np.zeros((9,16), dtype=int)
for p in skel:
    img[p[0],p[1]] = 1

plt.imshow(img, cmap='gray')

In [ ]:
I = io.imread(my_path+"blob2.png")

I1 = color.rgb2gray(I)
I1 = util.img_as_ubyte(I1)

J1 = I1==54
#etc

plt.subplot(3,4,1)
plt.imshow(I1, cmap='gray')

plt.subplot(3,4,2)
plt.imshow(J1, cmap='gray')

plt.show()

## Axe Median 

```
I : image, O son objet
d : transformée de distance de I
MA = I
Pour chaque pixel p de O
S’il existe q dans Voisins(p) tq d(q) > d(p)
MA(p) = 1 - I(p)
```

Pour itérer sur l'image en ayant accès aux **coordonnées** des pixels (et non uniquement à leur valeur) voir ici :

https://docs.scipy.org/doc/numpy/reference/arrays.nditer.html#tracking-an-index-or-multi-index

Concrètement, ça vous donne un code comme ça :
```
it = np.nditer(mon_image, flags=['multi_index'], op_flags=['readwrite']) 
while not it.finished:
    p = it.multi_index # coordonnées du pixel courant : p = (i, j)
    # votre code
    it.iternext()
it.close()
```

In [ ]:
# Boules maximales - distance "de Manhattan" (ou "taxicab", ou encore d1)
from scipy.ndimage import distance_transform_cdt as dt

def medial_axis(img, metric = 'taxicab'):
    medial_axis = img.copy()

    # votre code
    
    return medial_axis
    
    


img = io.imread("Images/A.png")

#ma = medial_axis(img)

plt.subplot(1,2,1)
plt.imshow( img, cmap='gray')

plt.subplot(1,2,2)
#plt.imshow(ma, cmap='gray')
plt.show()

### Autres squelettes dans skimage
Dans le module `skimage.morphology`, il y a trois algorithme de squelettisation homotopique, `medial_axis` (qui est basé sur la transformée de distance mais qui n'enlève que des pixels simples contrairement à l'axe médian standard), `skeletonize` (amincissement parallèle) et `thin` (amincissement homotopique en 8 adjacence).

In [ ]:
from skimage.morphology import medial_axis as m_a
from skimage.morphology import skeletonize as skl
from skimage.morphology import thin as thin
